In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import tqdm
import pdb
from scipy.sparse import csr_matrix, linalg
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb

import random
import os
import pandas

warnings.filterwarnings(action='ignore')

In [33]:
pd.set_option('display.max_row', 50)

path = '../data/'

train = pd.read_csv(path+'ksy_train_rating_10n.csv')
test = pd.read_csv(path+'ksy_test_rating_10n.csv')
submit = pd.read_csv(path + 'sample_submission.csv')

def rmse(real: list, predict: list) -> float:
    pred = np.array(predict)
    return np.sqrt(np.mean((real-pred) ** 2))

SEED = 42
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(SEED)

print(train.shape)
print(test.shape)

(306795, 10)
(76699, 10)


In [34]:
tem = train['user_id'].value_counts()
tem = list(tem[tem >= 0].index)
train['id'] = train['user_id'].copy()
train['id'][~train['id'].isin(tem)] = -1
test['id'] = test['user_id'].copy()
test['id'][~test['id'].isin(tem)] = -1

In [35]:
tem = train['isbn'].value_counts()
tem = list(tem[tem >= 0].index)
train['bn'] = train['isbn'].copy()
train['bn'][~train['bn'].isin(tem)] = 1000000000
test['bn'] = test['isbn'].copy()
test['bn'][~test['bn'].isin(tem)] = 1000000000

In [36]:
# train['years'] = train['years'].astype('str')
# train['fix_age'] = train['fix_age'].astype('str')
# train['id'] = train['id'].astype('str')
# train['bn'] = train['bn'].astype('str')

# test['years'] = test['years'].astype('str')
# test['fix_age'] = test['fix_age'].astype('str')
# test['id'] = test['id'].astype('str')
# test['bn'] = test['bn'].astype('str')

In [37]:
cat_features = ['isbn','book_author','publisher','language', 'category_high', 'fix_location_state', 'bn']
for i in enumerate (cat_features) :
    ca = i[1]
    train[ca] = train[ca].astype('category')
    test[ca] = test[ca].astype('category')


In [38]:
X_tr, X_val, y_tr, y_val = train_test_split(train.drop(['user_id', 'isbn', 'rating'],axis = 1), train['rating'], test_size=0.2)

In [39]:
train_ds = lgb.Dataset(X_tr, label = y_tr) 
test_ds = lgb.Dataset(X_val, label = y_val) 

In [40]:
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'rmse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':42}

In [41]:
model = lgb.train(params, train_ds, 2000, test_ds, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042810 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 17667
[LightGBM] [Info] Number of data points in the train set: 245436, number of used features: 9
[LightGBM] [Info] Start training from score 7.068886
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 2.34749
[200]	valid_0's rmse: 2.3286
[300]	valid_0's rmse: 2.32343
[400]	valid_0's rmse: 2.32085
[500]	valid_0's rmse: 2.31935
[600]	valid_0's rmse: 2.31897
[700]	valid_0's rmse: 2.31847
[800]	valid_0's rmse: 2.31784
[900]	valid_0's rmse: 2.31747
[1000]	valid_0's rmse: 2.31739
[1100]	valid_0's rmse: 2.3169
[1200]	valid_0's rmse: 2.31653
[1300]	valid_0's rmse: 2.31624
[1400]	valid_0's rmse: 2.31587
[1500]	valid_0's rmse: 2.31572
Early stopping, best iteration is:
[1474]	valid_0's rmse: 2.31561
